# Assignment 2 Part 2

Abhinav Kumar
10/13/2025

PatchCore on MVTec-AD

In [18]:
from pathlib import Path
import numpy as np, pandas as pd

from lightning import Trainer, seed_everything
from anomalib.data import MVTec
from anomalib.models import Patchcore

try:
    from anomalib.utils.callbacks.image_visualizer import ImageVisualizerCallback
except Exception:
    try:
        from anomalib.engine.callbacks import ImageVisualizerCallback
    except Exception:
        try:
            # older layout
            from anomalib.utils.callbacks import ImageVisualizerCallback
        except Exception:
            ImageVisualizerCallback = None


SEED = 42
CATEGORIES = ("tile", "leather", "grid")
IMAGE_SIZE = 256
BATCH_SIZE = 8
NUM_WORKERS = 4
MAX_EPOCHS = 1
OUTDIR = Path("./outputs_hertz"); OUTDIR.mkdir(parents=True, exist_ok=True)
seed_everything(SEED)


Seed set to 42


42

In [19]:
import inspect

def build_mvtec_dm(category: str,
                   root="./data",
                   train_batch_size=8,
                   eval_batch_size=8,
                   num_workers=4,
                   download=True):
   
    candidates = []
    try:
        from anomalib.data import MVTecAD
        candidates.append(MVTecAD)
    except Exception:
        pass
    try:
        from anomalib.data import MVTec
        candidates.append(MVTec)
    except Exception:
        pass
    if not candidates:
        raise RuntimeError("Could not import anomalib.data.MVTecAD or MVTec")

    desired = dict(
        root=root,
        category=category,
        train_batch_size=train_batch_size,
        eval_batch_size=eval_batch_size,
        num_workers=num_workers,
        download=download,
        task="segmentation",   
        image_size=256, 
    )

    last_err = None
    for cls in candidates:
        params = set(inspect.signature(cls.__init__).parameters.keys())
        params.discard("self")
        filtered = {k: v for k, v in desired.items() if k in params}
        try:
            dm = cls(**filtered)
            
            return dm
        except TypeError as e:
            last_err = e
            continue

    raise TypeError(f"Failed to construct MVTec datamodule; last error: {last_err}")


In [20]:
def build_dm(category: str):
    return build_mvtec_dm(
        category=category,
        root="./data",
        train_batch_size=BATCH_SIZE,
        eval_batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        download=True,
    )


In [21]:

def run_category(category: str):
    dm = build_dm(category)
    model = Patchcore()
    callbacks = [ImageVisualizerCallback(save_images=True, mode="full", output_path=OUTDIR / category)]
    trainer = Trainer(
        accelerator="cpu", devices=1, precision="32-true",
        max_epochs=MAX_EPOCHS, callbacks=callbacks,
        logger=False, enable_checkpointing=True, default_root_dir=str(OUTDIR / category),
    )
    trainer.fit(model=model, datamodule=dm)
    metrics_list = trainer.test(model=model, datamodule=dm, verbose=False)
    metrics = metrics_list[0] if isinstance(metrics_list, list) else metrics_list
    img_auc = metrics.get("image_auroc", metrics.get("AUROCImage", np.nan))
    pxl_auc = metrics.get("pixel_auroc", metrics.get("AUROCPixel", np.nan))
    return float(img_auc) if img_auc is not None else np.nan, float(pxl_auc) if pxl_auc is not None else np.nan


In [ ]:

rows = []
for cat in CATEGORIES:
    print(f"\n=== {cat.upper()} ===")
    ia, pa = run_category(cat)
    rows.append({"category": cat, "image_auroc": ia, "pixel_auroc": pa})
df = pd.DataFrame(rows).set_index("category")
df.to_csv(OUTDIR / "auroc_summary.csv")
display(df)
print(f"\nMean Image AUROC: {df['image_auroc'].mean():.4f}")
print(f"Mean Pixel AUROC: {df['pixel_auroc'].mean():.4f}")
print("\nArtifacts saved to:", OUTDIR.resolve())



=== TILE ===
